# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data.sqlite")

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [5]:
q = """
    SELECT
        e.firstName, e.lastName, o.city
    FROM
        employees as e JOIN offices as o
        USING(officeCode)
    WHERE
        o.city = "Boston"
    
    """

df = pd.read_sql(q, conn)
df

,firstName,lastName,city
0,Julie,Firrelli,Boston
1,Steve,Patterson,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [16]:
q1 = """
    SELECT
        o.officeCode,
        o.city, 
        COUNT(e.employeeNumber) AS num_employees
    FROM
        offices as o 
        LEFT JOIN employees as e
        USING(officeCode)
    GROUP BY 
        o.officeCode
    HAVING 
        num_employees = 0
    """

df = pd.read_sql(q1, conn)
df

,officeCode,city,num_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [22]:
# How many customers are there per office?

# Example question: 
q = """
    SELECT
        e.officeCode,
        o.city,
        COUNT(c.customerNumber) AS num_customer
    FROM
        offices as o
        JOIN employees as e
        USING(officeCode)
        JOIN customers as c
        ON c.salesRepEmployeeNumber = e.employeeNumber
    GROUP BY
        e.officeCode
    """

pd.read_sql(q, conn)


,officeCode,city,num_customer
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [23]:
"""
# How many customers are there per office?
"""

q = """
    SELECT
        e.officeCode,
        o.city,
        COUNT(c.customerNumber) AS num_customer
    FROM
        offices as o
        JOIN employees as e
        USING(officeCode)
        JOIN customers as c
        ON c.salesRepEmployeeNumber = e.employeeNumber
    GROUP BY
        e.officeCode
    """

pd.read_sql(q, conn)

,officeCode,city,num_customer
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [25]:
"""
How many customers does each employee have
"""

q = """
    SELECT 
        firstName, 
        lastName, 
        COUNT(customerNumber) as num_customers
    FROM
        employees as e
        JOIN customers as c
        ON c.salesRepEmployeeNumber = e.employeeNumber
    GROUP BY employeeNumber
        
    """
pd.read_sql(q, conn)

,firstName,lastName,num_customers
0,Leslie,Jennings,6
1,Leslie,Thompson,6
2,Julie,Firrelli,6
3,Steve,Patterson,6
4,Foon Yue,Tseng,7
5,George,Vanauf,8
6,Loui,Bondur,6
7,Gerard,Hernandez,7
8,Pamela,Castillo,10
9,Larry,Bott,8


In [27]:
"""
Which office has the most customers
"""

q = """
    SELECT 
        o.city,
        count(customerNumber) as num_customers
    FROM
        offices as o
        JOIN employees as e
        USING(officeCode)
        JOIN customers as c
        ON c.salesRepEmployeeNumber = e.employeeNumber
    GROUP BY o.city
        
    """
pd.read_sql(q, conn)

,city,num_customers
0,Boston,12
1,London,17
2,NYC,15
3,Paris,29
4,San Francisco,12
5,Sydney,10
6,Tokyo,5


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [44]:

q = """
    SELECT
        e.firstName,
        e.lastName,
        p.productName
    FROM
    employees as e
    JOIN customers as c
        ON c.salesRepEmployeeNumber = e.employeeNumber
    JOIN orders as o
        USING(customerNumber)
    JOIN orderdetails as od
        USING(OrderNumber)
    JOIN products as p
        USING(productCode)
    

        
    """
pd.read_sql(q, conn)

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [46]:
q = """
    SELECT
        e.firstName,
        e.lastName,
        count(p.productName) as num_product
    FROM
    employees as e
    JOIN customers as c
        ON c.salesRepEmployeeNumber = e.employeeNumber
    JOIN orders as o
        USING(customerNumber)
    JOIN orderdetails as od
        USING(OrderNumber)
    JOIN products as p
        USING(productCode)
    GROUP BY 
        e.employeeNumber
    

        
    """
pd.read_sql(q, conn)

,firstName,lastName,num_product
0,Leslie,Jennings,331
1,Leslie,Thompson,114
2,Julie,Firrelli,124
3,Steve,Patterson,152
4,Foon Yue,Tseng,142
5,George,Vanauf,211
6,Loui,Bondur,177
7,Gerard,Hernandez,396
8,Pamela,Castillo,272
9,Larry,Bott,236


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [50]:
q = """
    SELECT
        e.firstName,
        e.lastName,
        count(p.productName) as num_product
    FROM
    employees as e
    JOIN customers as c
        ON c.salesRepEmployeeNumber = e.employeeNumber
    JOIN orders as o
        USING(customerNumber)
    JOIN orderdetails as od
        USING(OrderNumber)
    JOIN products as p
        USING(productCode)
    GROUP BY 
        e.employeeNumber
    HAVING 
        num_product > 200
    ORDER BY
        num_product desc
        
    

        
    """
pd.read_sql(q, conn)

,firstName,lastName,num_product
0,Gerard,Hernandez,396
1,Leslie,Jennings,331
2,Pamela,Castillo,272
3,Larry,Bott,236
4,Barry,Jones,220
5,George,Vanauf,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!